# CMA-GFS

CMA-GFS 基础 GRIB2 数据文件有 861 个要素场。

In [3]:
import numpy as np
import pandas as pd
import xarray as xr

from reki.data_finder import find_local_file
from reki.format.grib.eccodes import load_field_from_file

In [4]:
start_time = pd.Timestamp.utcnow().floor(freq="D") - pd.Timedelta(days=2)
start_time_label = start_time.strftime("%Y%m%d%H")
forecast_time_label = "24h"
forecast_time = pd.to_timedelta(forecast_time_label)

gfs_grib2_orig_file_path = find_local_file(
    "cma_gfs_gmf/grib2/orig",
    start_time=start_time,
    forecast_time=forecast_time,
)
gfs_grib2_orig_file_path

PosixPath('/g3/COMMONDATA/OPER/CEMC/GFS_GMF/Prod-grib/2025072200/ORIG/gmf.gra.2025072200024.grb2')

## 要素列表

使用 ecCodes 打印 CMA-GFS 基础 GRIB2 数据的要素列表。

In [9]:
from IPython.utils.capture import capture_output

with capture_output() as captured:
    !module load eccodes/2.29.0/intel && grib_ls -P count "{gfs_grib2_orig_file_path}" 2>/dev/null

print(captured.stdout)

/g3/COMMONDATA/OPER/CEMC/GFS_GMF/Prod-grib/2025072200/ORIG/gmf.gra.2025072200024.grb2
count        edition      centre       date         dataType     gridType     typeOfLevel  level        stepRange    shortName    packingType  
1            2            babj         20250722     fc           regular_ll   surface      0            0-24         acpcp        grid_jpeg   
2            2            babj         20250722     fc           regular_ll   surface      0            0-24         ncpcp        grid_jpeg   
3            2            babj         20250722     fc           regular_ll   surface      0            0-24         unknown      grid_jpeg   
4            2            babj         20250722     fc           regular_ll   surface      0            0-24         asnow        grid_jpeg   
5            2            babj         20250722     fc           regular_ll   surface      0            24           t            grid_jpeg   
6            2            babj         20250722     fc 

使用 wgrib2 打印 CMA-GFS 基础 GRIB2 数据的要素列表。

In [10]:
from IPython.utils.capture import capture_output

with capture_output() as captured:
    !module load wgrib2/3.1.1/intel && wgrib2 "{gfs_grib2_orig_file_path}" 2>/dev/null

print(captured.stdout)

1:0:d=2025072200:ACPCP:surface:0-1 day acc fcst:
2:4649143:d=2025072200:NCPCP:surface:0-1 day acc fcst:
3:9885269:d=2025072200:APCP:surface:0-1 day acc fcst:
4:15811618:d=2025072200:ASNOW:surface:0-1 day acc fcst:
5:19460994:d=2025072200:TMP:surface:24 hour fcst:
6:22059970:d=2025072200:NLWRF:surface:0-1 day acc fcst:
7:26178745:d=2025072200:NSWRF:surface:0-1 day acc fcst:
8:30353344:d=2025072200:HFLUX:surface:0-1 day acc fcst:
9:32812562:d=2025072200:LHTFL:surface:0-1 day acc fcst:
10:35534283:d=2025072200:ULWRF:surface:0-1 day acc fcst:
11:39100075:d=2025072200:ULWRF:top of atmosphere:0-1 day acc fcst:
12:42993694:d=2025072200:DLWRFCS:surface:0-1 day acc fcst:
13:45693544:d=2025072200:var discipline=0 center=38 local_table=1 parmcat=5 parm=224:surface:0-1 day acc fcst:
14:49256746:d=2025072200:var discipline=0 center=38 local_table=1 parmcat=5 parm=224:top of atmosphere:0-1 day acc fcst:
15:52062058:d=2025072200:USWRF:surface:0-1 day acc fcst:
16:55185583:d=2025072200:NSWRF:top of at

对 wgrib2 输出信息进行分组展示。

```{note}
本节代码由 ChatGPT 生成。
```

In [20]:
import re
import textwrap
from collections import defaultdict
from IPython.utils.capture import capture_output

with capture_output() as captured:
    !module load wgrib2/3.1.1/intel && wgrib2 "{gfs_grib2_orig_file_path}" 2>/dev/null

text = captured.stdout

lines = text.strip().splitlines()
grouped = defaultdict(list)

pattern = re.compile(r'^(\d+):\d+:d=\d+:([^:]+):([^:]+):([^:]+):?$')

for line in lines:
    match = pattern.match(line)
    if match:
        index, param, level, fcst = match.groups()
        grouped[(param.strip(), fcst.strip())].append((level.strip(), int(index)))

WRAP_WIDTH = 80

for (param, fcst), level_index_list in grouped.items():
    level_index_list.sort(key=lambda x: x[1])
    level_strs = [f"{level} ({index})" for level, index in level_index_list]
    full_line = ', '.join(level_strs)
    wrapped = textwrap.fill(full_line, width=WRAP_WIDTH, subsequent_indent='  ', initial_indent='  ')
    print(f"{param} @ {fcst}:\n{wrapped}\n")

ACPCP @ 0-1 day acc fcst:
  surface (1)

NCPCP @ 0-1 day acc fcst:
  surface (2)

APCP @ 0-1 day acc fcst:
  surface (3)

ASNOW @ 0-1 day acc fcst:
  surface (4)

TMP @ 24 hour fcst:
  surface (5), 2 m above ground (25), 1000 mb (104), 975 mb (105), 950 mb (106),
  925 mb (107), 900 mb (108), 850 mb (109), 800 mb (110), 750 mb (111), 700 mb
  (112), 650 mb (113), 600 mb (114), 550 mb (115), 500 mb (116), 450 mb (117),
  400 mb (118), 350 mb (119), 300 mb (120), 275 mb (121), 250 mb (122), 225 mb
  (123), 200 mb (124), 175 mb (125), 150 mb (126), 125 mb (127), 100 mb (128),
  70 mb (129), 50 mb (130), 30 mb (131), 20 mb (132), 10 mb (133), 7 mb (134), 5
  mb (135), 4 mb (136), 3 mb (137), 2 mb (138), 1.5 mb (139), 1 mb (140), 0.5 mb
  (141), 0.2 mb (142), 0.1 mb (143), 0-0.1 m below ground (464), 0.1-0.4 m below
  ground (465), 0.4-1 m below ground (466), 1-2 m below ground (467)

NLWRF @ 0-1 day acc fcst:
  surface (6)

NSWRF @ 0-1 day acc fcst:
  surface (7), top of atmosphere (16)

H